In [1]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

In [6]:
# A bit of customisation
matplotlib.rcParams['figure.figsize'] = (18.0, 5.0)

# %matplotlib inline
%config InlineBackend.figure_format='retina'

np.set_printoptions(precision=3, suppress=True)  # suppress scientific float notation

In [2]:
path = r'/emhires-data/data-warehouse/tabular/ERA-NUTS/'
# path = '.'

# Load data
Load all the files as a single dataset using xarray and its dask capabilities. In this way the files are not loaded in-memory but dask/xarray load them **only** when needed. `mask_and_scale` to True replaces array values equal to `_FillValue` with NA and scale values according to the formula `original_values * scale_factor + add_offset`.


In [28]:
ds = xr.open_mfdataset(path + 'NUTS2/hourly/*.nc', mask_and_scale=True)
ds

<xarray.Dataset>
Dimensions:  (region: 309, time: 341880)
Coordinates:
  * time     (time) datetime64[ns] 1980-01-01 ... 2018-12-31T23:00:00
  * region   (region) object 'AL01' 'AL02' 'AL03' ... 'UKM8' 'UKM9' 'UKN0'
Data variables:
    CS       (time, region) float32 dask.array<shape=(341880, 309), chunksize=(341880, 309)>
    ssrd     (time, region) float32 dask.array<shape=(341880, 309), chunksize=(341880, 309)>
    ssrdc    (time, region) float32 dask.array<shape=(341880, 309), chunksize=(341880, 309)>
    t2m      (time, region) float32 dask.array<shape=(341880, 309), chunksize=(341880, 309)>
    ws10     (time, region) float32 dask.array<shape=(341880, 309), chunksize=(341880, 309)>
    ws100    (time, region) float32 dask.array<shape=(341880, 309), chunksize=(341880, 309)>

# Widget

The above selection method allows as to build custom interactive dashboards with a simple jupyter widgets through `ipywidget`. This process is very is simple. You just need to encapsulate you query in a function and decorate it with [interact](https://ipywidgets.readthedocs.io/en/stable/examples/Using%20Interact.html#Basic-interact) by specifying the desired range. Due to xarray and dask's out of core computation, querying and plotting this dataset is almost instant

In [4]:
from ipywidgets import interact

In [29]:
@interact(var=['t2m','ssrd','ssrdc','ws10','ws100','CS'], 
          region=ds.coords['region'].values,
          month=(1,12),
          year=(1980,2018))
def plot_region_month(var, region, year, month):
    ds_hourly[var].sel(time='{}-{}'.format(year, month), region=region).plot(color='k') # year

interactive(children=(Dropdown(description='var', options=('t2m', 'ssrd', 'ssrdc', 'ws10', 'ws100', 'CS'), val…

In [33]:
@interact(var=['t2m','ssrd','ws10','ws100','CS'], 
          region=ds.coords['region'].values,
          from_year=(1980,2018),
          to_year=(1980,2018)
          )
def plot_region_month(var, region, from_year, to_year):
    data_sel = ds[var].sel(region = region)
    data_sel.sel(time=slice('{}'.format(from_year), '{}'.format(to_year))).resample(time = 'M').mean().plot(color='k') # year

interactive(children=(Dropdown(description='var', options=('t2m', 'ssrd', 'ssrdc', 'ws10', 'ws100', 'CS'), val…

Querying works also for a selection of regions

In [34]:
#Extract unique countries by taking the first two characters of all regions
countries = set(s[0:2] for s in ds.coords['region'].values)

@interact(var=['t2m','ssrd','ssrdc','ws10','ws100','CS'], 
          region=countries,
          month=(1,12),
          year=(1980,2018))
def plot_region_month(var, region, year, month):
    selection = [s for s in ds.coords['region'].values if s[0:2]==region]
    ds[var].sel(time='{}-{}'.format(year, month), region=selection).plot.line(x='time')

interactive(children=(Dropdown(description='var', options=('t2m', 'ssrd', 'ssrdc', 'ws10', 'ws100', 'CS'), val…